# Dobot Magician Calibration Tutorial

Calibrating the Dobot Magician with the camera is essential for accurate position and interaction with the real world environment. This tutorial will guide you through the eye-to-hand calibration process.

## Prerequisites

1. Completed the tutorial on [Dobot Magician Robot Arm Control](dobot.ipynb).
2. Basic understanding of Python programming.
3. Basic knowledge of linear algebra and transformations.
4. Intel RealSense camera installed for the full workspace view.
5. AprilTag installed on the robot arm (above the gripper).

## Learning Objectives
1. Understand the concept of eye-to-hand calibration.
2. Learn how to perform eye-to-hand calibration using the Dobot Magician and Intel RealSense camera.
3. Apply the calibration to improve the accuracy of the robot arm's movements.

## Outcome
By the end of this tutorial, you will be able to calibrate the Dobot Magician with the Intel RealSense camera, write the transformation matrix from the robot base to the end effector, and verify the calibration by moving the robot arm to the AprilTag position.

## Key NumPy Functions for Working with Robot Transformations

When working with robot pose data and transformation matrices, several NumPy functions are commonly used:

- **`np.arctan2(y, x)`**: Computes the angle (in radians) from the X-axis to the point $(x, y)$. This is useful for determining the orientation of the robot or its end effector in the plane.

- **`np.array([...])`**: Creates a NumPy array from a Python list or nested lists. Arrays are used to represent vectors (such as position) and matrices (such as rotation).

- **`np.eye(n)`**: Generates an $n \times n$ identity matrix. This is often used as a starting point for building transformation matrices, since the identity matrix represents no rotation or translation.

### Example Usage

- To create a rotation matrix for a planar robot, you might use `np.arctan2` to get the angle, then use `np.array` to build the matrix.
- To represent the robot's position, use `np.array([x, y, z])`.
- To start with a $4 \times 4$ identity matrix for a transformation, use `np.eye(4)`.

Now is your turn to practice these concepts! Write the code to get the transformation matrix from the robot base to the end effector. Run the test to verify your implementation.

## Mathematical Construction of the Transformation Matrix

The transformation from one coordinate frame to another is represented by a 4x4 matrix:

$$
T = \begin{bmatrix}
R & t \\
0 & 1
\end{bmatrix}
$$

Where:
- $R$ is the $3 \times 3$ rotation matrix (describes orientation)
- $t$ is the $3 \times 1$ translation vector (describes position)
- The bottom row $[0 \quad 1]$ ensures the matrix is suitable for homogeneous coordinates.

For example, to construct the transformation from the AprilTag to the camera frame:

$$
T_{\text{tag} \to \text{camera}} = \begin{bmatrix}
R_{\text{tag} \to \text{camera}} & t_{\text{tag} \to \text{camera}} \\
0 & 1
\end{bmatrix}
$$


## Get the Robot Base to End Effector Transformation Matrix

The transformation matrix from the robot base to the end effector can be constructed directly using the pose information provided by the robot's sensors.

### Steps to Construct the Transformation Matrix

1. **Pose Acquisition**: The robot controller or API typically provides the current pose of the end effector, including its position $(x, y, z)$ and orientation (often as a rotation matrix or quaternion).

2. **Matrix Construction**: Combine the position and orientation into a $4 \times 4$ homogeneous transformation matrix:
   $$
   T_{\text{base} \to \text{ee}} = \begin{bmatrix}
   R & t \\
   0 & 1
   \end{bmatrix}
   $$
   - $R$ is the $3 \times 3$ rotation matrix representing the end effector's orientation.
   - $t$ is the $3 \times 1$ translation vector $[x, y, z]^T$ representing the end effector's position.

## Practice 1

In [ ]:
import numpy as np
from pydobotplus import Dobot

def get_robot_base_to_ee(pose):
    """
    This function returns the transformation matrix from the robot base to the end effector.
    The transformation matrix is a 4x4 matrix that includes both rotation and translation.

    Args:
        pose (dict): A dictionary containing the robot's pose data, including position and joint information.
                    This includes 'position' (a 3-element list) - x, y, z coordinates

    Returns:
        np.ndarray: A 4x4 transformation matrix representing the pose of the end effector
                    in the robot base frame.
    """
    # Your implementation here
    return np.eye(4) # Edit this line to return the actual transformation matrix

print("Transformation Matrix from Robot Base to End Effector:")
port = "/dev/ttyACM0" # Edit this line to match your Dobot's port
dobot = Dobot(port=port)
T_base_to_ee = get_robot_base_to_ee(dobot.get_pose())
print(T_base_to_ee)

## Find the Transformation Matrix from AprilTag to End-effector

The transformation matrix from the gripper (end effector) to the AprilTag describes the position and orientation of the AprilTag relative to the gripper's coordinate frame. When the x-axes of both frames are aligned, you can measure the translation and orientation offsets using a ruler or caliper.

- The first row means the tag's x-axis is anti-aligned with the gripper's x-axis, and the tag is 30 mm away along x.
- The second row means the y-axes are aligned, with no offset.
- The third row means the tag's z-axis is opposite to the gripper's z-axis, and the tag is 153 mm away along z.
- The fourth row ensures homogeneous coordinates.

This matrix (`gHt`) allows you to transform points between the gripper and AprilTag frames. The tag size (0.792) is used for calibration and pose estimation.

## Practice 2

In [ ]:
# Edit this matrix based on your measurement
gHt = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype=np.float32)

# AprilTag & Camera
## Step 1: Turn on the Intel RealSense Camera
Intel RealSense camera is a depth camera that provides RGB frame and depth frame.   

To turn on the camera, you should do the following:
1. Connect the Intel RealSense camera to your computer.
2. Run the code to check the camera serial (Important step if you have multiple cameras connected)
3. Run the code to start the camera stream and display the RGB frame.

In [ ]:
import pyrealsense2 as rs

# Check the camera serial
ctx = rs.context()
serials = [device.get_info(rs.camera_info.serial_number) for device in ctx.query_devices()]
for i, serial in enumerate(serials):
    print(f"Camera Serial {i}: {serial}")

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
import pyrealsense2 as rs

camera_serial = "218622277520" # Replace with your camera serial

# Initialize the camera pipeline
ctx = rs.context()
pipeline = rs.pipeline(ctx)
config = rs.config()
config.enable_device(camera_serial)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
profile = pipeline.start(config)
align = rs.align(rs.stream.color)

# Display a live camera stream
try:
    for _ in range(100):  # Show 100 frames, adjust as needed
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        if color_frame:
            color_image = np.asanyarray(color_frame.get_data())
            plt.imshow(cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB))
            plt.title("RGB Frame")
            plt.axis('off')
            clear_output(wait=True)
            display(plt.gcf())
            plt.clf()
finally:
    pipeline.stop()

Now, you should see the live camera stream.  

## Step 2: Find the AprilTag
AprilTag is a visual fiducial system that allows the robot arm to recognize its position in the workspace. The AprilTag is attached to the robot arm above the gripper.
You can use the `pupil-apriltags` library to detect the AprilTag in the camera frame. The library provides functions to detect AprilTags and get their positions in the camera coordinate system.

In [ ]:
from pupil_apriltags import Detector
import cv2

import matplotlib.pyplot as plt

# Initialize AprilTag detector
detector = Detector()

# Convert color image to grayscale for AprilTag detection
gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

# Detect AprilTags in the grayscale image
tags = detector.detect(gray_image, estimate_tag_pose=False, camera_params=None, tag_size=None)

# Draw detected tags on the image
image_with_tags = color_image.copy()
for tag in tags:
    corners = tag.corners.astype(int)
    for i in range(4):
        pt1 = tuple(corners[i])
        pt2 = tuple(corners[(i + 1) % 4])
        cv2.line(image_with_tags, pt1, pt2, (0, 255, 0), 2)
    # Draw center
    center = tuple(tag.center.astype(int))
    cv2.circle(image_with_tags, center, 5, (0, 0, 255), -1)
    # Put tag ID
    cv2.putText(image_with_tags, str(tag.tag_id), center, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

# Display the image
plt.imshow(cv2.cvtColor(image_with_tags, cv2.COLOR_BGR2RGB))
plt.title("AprilTag Detection")
plt.axis('off')
plt.show()

## Finding the Transformation from Camera to AprilTag
The first step of building this connection is to find the transformation matrix from the camera to the AprilTag. This is done by detecting the AprilTag in the camera frame and calculating its position and orientation. Thanks to the `pupil-apriltags` library, you can easily detect the AprilTag and get its translation vector and rotation matrix.

To construct the transformation matrix, you will need to combine the translation vector and rotation matrix into a single transformation matrix. The translation vector is the position of the AprilTag in the camera coordinate system, and the rotation matrix is the orientation of the AprilTag in the camera coordinate system.  

## Practices 3

In [ ]:
from pupil_apriltags import Detector
import cv2
import numpy as np

# Get camera intrinsics from RealSense2
intr = profile.get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
camera_params = [intr.fx, intr.fy, intr.ppx, intr.ppy]

tag_size = 0.0792  # Tag size in meters (!! Measure this by yourself)

# Initialize AprilTag detector
detector = Detector()

# Convert color image to grayscale for AprilTag detection
gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

# Detect AprilTags and estimate pose
tags = detector.detect(
    gray_image,
    estimate_tag_pose=True,
    camera_params=camera_params,
    tag_size=tag_size
)

for tag in tags:
    print(f"Tag ID: {tag.tag_id}")
    print("Translation (x, y, z) in meters:", tag.pose_t)
    print("Rotation matrix (3x3):\n", tag.pose_R)
    # You can construct the 4x4 transformation matrix 
    # HERE!!!!!
    print("Transformation matrix (tag to camera):\n", T_tag_to_camera)

# Step 3: Calculate the transformation matrix

Let's break down how to connect the robot base and the camera using simple steps.

Imagine you want to know where the camera is compared to the robot base. You can't measure this directly, but you can use a chain of connections:

1. **Robot base to end effector**: This is the position of the robot's moving part (the end effector, like the gripper) relative to the robot base. The robot can tell you this using its own sensors.
2. **End effector to AprilTag**: The AprilTag is attached to the end effector. You measure how far and in what direction the tag is from the end effector. This is usually a fixed value, since you attach the tag yourself.
3. **AprilTag to camera**: The camera sees the AprilTag and can calculate where the tag is in its own view.

To get the full connection from the robot base to the camera, you combine these three steps:

$$\text{Robot base to camera} = \text{Robot base to end effector} \times \text{End effector to AprilTag} \times \text{AprilTag to camera}$$

Or, using symbols:

$$T_{\text{base} \to \text{camera}} = T_{\text{base} \to \text{ee}} \times T_{\text{ee} \to \text{tag}} \times T_{\text{tag} \to \text{camera}}$$

![Calibration Diagram](Screenshot%20from%202025-07-14%2010-47-33.png)

# Final Steps: Construct the robot base to camera transformation matrix

To construct the final transformation matrix from the robot base to the camera, you will need to combine the previously calculated matrices:

$$
T_{base}^{camera} =  T_{AprilTag}^{camera} \cdot T_{gripper}^{AprilTag}  \cdot T_{base}^{gripper}
$$

Where:
- $T_{base}^{gripper}$ is the transformation matrix from the robot base to the end effector.
- $T_{gripper}^{AprilTag}$ is the transformation matrix from the end effector to the AprilTag.
- $T_{AprilTag}^{camera}$ is the transformation matrix from the AprilTag to the camera.
- $T_{base}^{camera}$ is the final transformation matrix from the robot base to the camera.


### Essential NumPy Functions for Matrix Calculations

To combine transformation matrices and perform robot calibration, you will use the following NumPy functions:

- **`np.dot(A, B)`** or **`A @ B`**: Matrix multiplication. Use this to multiply transformation matrices in the correct order.
- **`np.linalg.inv(A)`**: Computes the inverse of a matrix. Useful if you need to reverse a transformation (e.g., from camera to AprilTag).
- **`np.eye(n)`**: Creates an identity matrix of size $n \times n$. Often used to initialize transformation matrices.
- **`np.array([...])`**: Converts a Python list to a NumPy array, which is required for matrix operations.

#### Example: Matrix Multiplication

To combine the matrices for the final transformation:

```python
T_base_to_camera = T_base_to_ee @ gHt @ T_tag_to_camera
```

This multiplies the matrices in the correct order to get the transformation from the robot base to the camera.

#### Example: Inverse Transformation

If you need the transformation from camera to robot base:

```python
T_camera_to_base = np.linalg.inv(T_base_to_camera)
```

Now, try to write the code to get the transformation matrix from the robot base to the camera. Use the provided functions and matrices to perform the calculations. Once you have your implementation, run it on the robotics arm to verify the formula.

In [ ]:
def get_robot_base_to_camera(T_base_to_ee, T_ee_to_tag, T_tag_to_camera):
    # Your implementation here
    T_base_to_camera = np.eye(4) # Edit this line to return the actual transformation matrix
    return T_base_to_camera

# Verifying the Transformation Matrix

To verify the transformation matrix, we can use the apriltag detection and the camera stream to check if the robot arm can accurately reach the AprilTag position.  

Now, paste your code to `calibration/utils.py` and run the code to verify the transformation matrix. 

First, use command `python3 scripts/check_port.py` to check the dobot port and the camera serial. This will ensure that the robot arm and camera are connected properly.

Use command `python3 calibration/calibration_simplified.py` to run the data collection script. This script will collect the transformation matrices from the robot base to the camera for multiple AprilTag detections and calculate the average transformation matrix.

Use command `python3 calibration/calibration_validate.py` to run the validation script. This script will use the average transformation matrix to validate the calibration by moving the robot arm to the AprilTag position and checking if it can reach it accurately.

If your function writes the transformation matrix correctly, the robot arm should be able to reach the AprilTag position accurately. If not, you may need to adjust the transformation matrix formula.